In [1]:
import requests
import time

In [2]:
def request_url(url):
    response = requests.get(url)
    while(response.status_code != 200):
        response = requests.get(url)
        time.sleep(1)
    response = response.json()
    return response

In [3]:
def get_discover_tracks_id(CLIENT_ID):
    result = set()
    
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while(1):
        response = request_url(url)
        collections = response['collection']    
        for collection in collections:
            items = collection['items']['collection']
            for item in items:
                try:
                    tracks = item['tracks']
                    for track in tracks:
                        result.add(track['id'])
                except:
                    break
        try:
            url = response['next_href']+ '&client_id='+CLIENT_ID
        except:
            break
    return result

In [4]:
CLIENT_ID='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'

result=get_discover_tracks_id(CLIENT_ID)
len(result)

3410

In [5]:
def get_discover_playlists_id(CLIENT_ID):
    result = []
    temp=set()
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while(1):
        response = request_url(url)
        collections = response['collection']    
        for collection in collections:
            items = collection['items']['collection']
            for item in items:
                try:
                    uri = item['uri']
                    temp.add(uri)
                except:
                    break
        try:
            url = response['next_href']+ '&client_id='+CLIENT_ID
        except:
            break
    new_temps=list(temp)
    for new_temp in new_temps:
        result.append(new_temp[37:])
    return result

In [6]:
CLIENT_ID='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'

result = get_discover_playlists_id(CLIENT_ID)

In [7]:
len(result)

212

In [8]:
#user_id
#https://api-v2.soundcloud.com/tracks/591945735?client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&limit=100&linked_partitioning=1
#https://api-v2.soundcloud.com/tracks/591945735/related?client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&limit=100&linked_partitioning=1

#https://api.soundcloud.com/users/
#https://api-v2.soundcloud.com/tracks/591945735/likers?client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&limit=100&linked_partitioning=1
#https://api-v2.soundcloud.com/tracks/591945735/playlists_without_albums?client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&limit=100&linked_partitioning=1
#https://api-v2.soundcloud.com/tracks/197304661/albums?&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&limit=100&linked_partitioning=1
#https://api-v2.soundcloud.com/tracks/591945735/reposters?client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&limit=100&linked_partitioning=1(*)
#https://api-v2.soundcloud.com/tracks/591945735/comments?threaded=0&filter_replies=0&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&limit=100&linked_partitioning=1(*)


In [53]:
def get_track_url(TRACK_ID,CLIENT_ID,option):
    head = 'https://api-v2.soundcloud.com/tracks/'+ TRACK_ID
    tail = 'client_id=' + CLIENT_ID + '&limit=100&linked_partitioning=1'
    middle =''
    if(option == 'comments'):
        middle = 'threaded=0&filter_replies=0&'
    if(option):
        option = '/'+option+'?'
    else:
        option = '?'
      
    url = head + option+ middle + tail
    return url

In [58]:
def get_track_user(TRACK_ID,CLIENT_ID,option):
    result=set()
    url = get_track_url(TRACK_ID,CLIENT_ID,option)
    if(option):   
        while(1):
            response = request_url(url)
            collections = response['collection']
            if(option=='comments'):
                for collection in collections:
                    result.add(collection['user']['id'])
            else:
                for collection in collections:
                    result.add(collection['id'])
            try:
                url = response['next_href']+ '&client_id='+CLIENT_ID
            except:
                break
    else:
        response = request_url(url)
        result.add(response['user']['id'])
    return result

In [62]:
CLIENT_ID='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'
TRACK_ID='591945735'
option =''

result = get_track_user(TRACK_ID,CLIENT_ID,option)
result

{587430921}

In [9]:
def track_info(TRACK_ID,CLIENT_ID):
    #result = {}
    #creator
    creator=get_track_user(TRACK_ID,CLIENT_ID,'')
    reposters=get_track_user(TRACK_ID,CLIENT_ID,'reposters')
    likers=get_track_user(TRACK_ID,CLIENT_ID,'likers')
    comments = get_track_user(TRACK_ID,CLIENT_ID,'comments')
    return creator,reposters,likers,comments